# Section 3
## Read in database credentials
Below the credentials for connecting to the database are read into variables by extracting the lines from the local file. The local file is not included in the repo.

In [360]:
db_name = ""
db_user = ""
db_pass = ""
db_host = ""
with open("database_credentials.txt") as f:
    db_name = f.readline().strip()
    db_user = f.readline().strip()
    db_pass = f.readline().strip()
    db_host = f.readline().strip()

## Shorthand connect to database
This method will just return a new database connection with the default credentials made available above.

In [361]:
import pymysql as pms

In [362]:
def get_connect():
    """
    Returns a database connection object using the default parameters
    specified in the database_credentials file read in at the start of
    this notebook.
    """
    return pms.connect(host=db_host, user=db_user, passwd=db_pass, db=db_name);

## Test connection
The next code segment tests to ensure that the database connection is working properly.

In [363]:
try:
    con = get_connect()
    print("Successfully connected")
finally:
    if con:
        print("Closing connection")
        con.close()

Successfully connected
Closing connection


In [364]:
def get_connect():
    """
    Returns a database connection object using the default parameters
    specified in the database_credentials file read in at the start of
    this notebook.
    """
    return pms.connect(host=db_host, user=db_user, passwd=db_pass, db=db_name);

## Shorthand query execution and output
The method below accepts a single parameter (expected query), executes the parameter as a SQL query, and outputs the results. The connection is closed before the function terminates.

In [365]:
def execute_sql_output_result(query_string, is_query=True):
    """
    Given the query_string parameter, this function connects to the database, executes
    the query, outputs the result, and closes the connection.
    """
    try:
        con = get_connect()
        with con.cursor() as cur:
            #If the query_string is a single string, execute the string
            if type(query_string) == str:
                cur.execute(query_string)
                #Only output query results if this is a query
                if is_query:
                    result = cur.fetchall()
                    print("=== {} RESULTS ===".format(len(result)))
                    #Column names
                    print(" ".join([i[0] for i in cur.description]))
                    #Results
                    for i in range(len(result)):
                        print("{}: {}".format(i, result[i]))
                    print()
    finally:
        if con:
            con.close()

## Shorthand statement execution (INSERT, CREATE, DROP)

In [366]:
def execute_sql_statement(statement):
    """
    Given the query_string parameter, this function connects to the database, executes
    the insertion, and closes the connection.
    """
    try:
        con = get_connect()
        with con.cursor() as cur:
            con.autocommit(True)
            #If the query_string is a single string, execute the string
            if type(statement) == str:
                cur.execute(statement)
    finally:
        if con:
            con.close()

## TABLE creation

In [367]:
execute_sql_statement("""
    CREATE TABLE IF NOT EXISTS stores (
        store_id INT NOT NULL,
        city VARCHAR(50)
    );
""")

## Inserting data

In [368]:
#Individual insert
execute_sql_statement("""
    INSERT INTO stores(store_id, city) VALUES (1, 'San Francisco');
    INSERT INTO stores(store_id, city) VALUES (2, 'New York');
    INSERT INTO stores(store_id, city) VALUES (3, 'CHICAGO');
""")
execute_sql_statement("""
    INSERT INTO stores(store_id, city) VALUES
        (4, 'Philadelphia'),
        (5, 'Boston'),
        (6, 'Seattle');
""")

In [369]:
execute_sql_output_result("""
    SELECT * FROM stores;
""")

=== 6 RESULTS ===
store_id city
0: (1, 'San Francisco')
1: (2, 'New York')
2: (3, 'CHICAGO')
3: (4, 'Philadelphia')
4: (5, 'Boston')
5: (6, 'Seattle')



## PRIMARY KEY
A primary key ensures uniqueness in a table.

In [370]:
execute_sql_statement("""
    CREATE TABLE IF NOT EXISTS products (
        product_id INT NOT NULL,
        name VARCHAR(50),
        product_cost FLOAT(5,2),
        product_retail FLOAT(5,2),
        product_type VARCHAR(10),
        store_id INT NOT NULL,
        PRIMARY KEY (product_id)
    );
    INSERT INTO products (product_id, name, product_cost, product_retail, product_type, store_id) VALUES
        (1001, 'Colgate Toothpaste', 2.25, 5.47, 'hygiene', 2),
        (1002, 'Colgate Toothpaste', 2.25, 5.47, 'hygiene', 2),
        (1003, 'Listerine Mouthwash', 1.75, 4.81, 'hygiene', 3),
        (1011, 'Listerine Sweatpants', 4.00, 8.00, 'Clothing', 4);
""")

In [371]:
execute_sql_output_result("""
    SELECT * FROM products;
""")

=== 4 RESULTS ===
product_id name product_cost product_retail product_type store_id
0: (1001, 'Colgate Toothpaste', 2.25, 5.47, 'hygiene', 2)
1: (1002, 'Colgate Toothpaste', 2.25, 5.47, 'hygiene', 2)
2: (1003, 'Listerine Mouthwash', 1.75, 4.81, 'hygiene', 3)
3: (1011, 'Listerine Sweatpants', 4.0, 8.0, 'Clothing', 4)



## Modifying the structure of a table using ALTER

In [372]:
execute_sql_statement("""
    ALTER TABLE products
        MODIFY name VARCHAR(50) NOT NULL,
        MODIFY product_retail FLOAT(5,2) NOT NULL;
""")

## Renaming a column

In [373]:
execute_sql_statement("""
    ALTER TABLE products
        CHANGE COLUMN name product_name VARCHAR(50) NOT NULL;
""")

## Creating another table from a SELECT

In [374]:
execute_sql_statement("""
    CREATE TABLE employees AS
        SELECT empno, ename, job, hiredate, sal, comm
    FROM emp;
""")

## ADD

In [375]:
execute_sql_statement("""
    ALTER TABLE employees
        ADD store_id INT;
""")

## UPDATE

In [376]:
execute_sql_statement("""
    UPDATE employees
        SET store_id = 3 WHERE
            ename IN ('KING', 'BLAKE', 'CLARK');
    UPDATE employees
        SET store_id = 2 WHERE
            job = 'SALESMAN';
    UPDATE employees
        SET store_id = 4 WHERE
            job IN ('CLERK', 'ANALYST');
    UPDATE employees
        SET store_id = 1 WHERE
            job = 'MANAGER';
    ALTER TABLE employees
        MODIFY store_id INT NOT NULL;
""")

## Dropping tables

In [359]:
execute_sql_statement("""
    DROP TABLE IF EXISTS stores;
    DROP TABLE IF EXISTS products;
    DROP TABLE IF EXISTS employees;
""")